In [1]:
import numpy as np
import pandas as pd
import random as rand

In [2]:
def weighted_random_question(agreement_val, correct_ans):
    my_list = []
    for i in np.arange(1, 6):
        if i == correct_ans:
            my_list += [str(correct_ans)] * int(agreement_val * 100)
        else:
            my_list += [str(i)] * int((1 - agreement_val) / abs(correct_ans - i) * 100)
    return int(rand.choice(my_list))

In [3]:
def weighted_random_pos(agreement_val, correct_ans, source_length):
    pos_range = int((1 - agreement_val) * source_length)
    start = correct_ans - pos_range
    end = correct_ans + pos_range + 1
    if start < 0:
        start = 0
    if end > source_length:
        end = source_length
    interval = np.arange(start, end)
    return rand.choice(interval)

In [4]:
def generate_agreement(agreement, num_users, num_questions, source_len):
    answer_number= np.zeros(0)
    start_pos = np.zeros(0)
    end_pos = np.zeros(0)
    for i in np.arange(num_questions):
        correct_ans = rand.randint(1, 5)
        correct_start = rand.randint(0, 200)
        correct_end = rand.randint(correct_start, 200)
        for j in np.arange(num_users):
            answer_number = np.append(answer_number, weighted_random_question(agreement[j], correct_ans))
            start_pos = np.append(start_pos, weighted_random_pos(agreement[j], correct_start, source_len))
            end_pos = np.append(end_pos, weighted_random_pos(agreement[j], correct_end, source_len))
    return answer_number, start_pos, end_pos

In [5]:
def create_run(agreement, num_run, num_users = 5, num_questions = 10, source_len = 200, ans_type = 'nominal'):
    rows = np.zeros(num_users * num_questions)
    inputs = generate_agreement(agreement, num_users, num_questions, source_len)
    d = {'taskrun_article_number': rows + num_run, 
         'contributor_id': np.tile(np.arange(1, num_users + 1), num_questions), 
         'question_number': np.repeat(np.arange(1, num_questions + 1), num_users), 
         'answer_number': inputs[0], 'start_pos': inputs[1], 
         'end_pos': inputs[2], 'source_text_length': rows + source_len, 
         'answer_type': np.repeat(ans_type, len(rows)), 'question_text': rows}
    run = pd.DataFrame(d)
    run = run[['taskrun_article_number', 'contributor_id', 
         'question_number', 'answer_number', 'start_pos', 
         'end_pos', 'source_text_length', 'answer_type', 'question_text']]
    return run

In [ ]:
current = create_run([0.9, 1, 1, 1, 1], 4)
current

In [6]:
current.to_csv('Rep_Runs/Run3.csv')

NameError: name 'current' is not defined

In [7]:
def create_series(agreement_arr):
    for i in np.arange(len(agreement_arr)):
        current = create_run(agreement_arr[i], i)
        current.to_csv('Series_Output/Run' + str(i) + '.csv')

In [ ]:
create_series([[0.9, 0.9, 0.9, 0.5, 0.1], [0.9, 0.9, 0.9, 0.5, 0.2], [0.9, 0.9, 0.9, 0.6, 0.3], 
               [0.9, 0.9, 0.9, 0.7, 0.4], [0.9, 0.9, 0.9, 0.8, 0.5], [0.9, 0.9, 0.9, 0.9, 0.5], 
               [0.9, 0.9, 0.9, 0.1, 0.6], [0.9, 0.9, 0.9, 0.1, 0.7], [0.9, 0.9, 0.9, 0.1, 0.8], 
               [0.9, 0.9, 0.9, 0.1, 0.9]])

In [18]:
create_series([[0.7, 0.7, 0.7, 0.7, 0.1], [0.7, 0.7, 0.7, 0.7, 0.5], [0.7, 0.7, 0.7, 0.7, 0.6], 
              [0.7, 0.7, 0.7, 0.7, 0.7], [0.7, 0.7, 0.7, 0.7, 0.8], [0.7, 0.7, 0.7, 0.7, 0.9],
              [0.7, 0.7, 0.7, 0.7, 0.9], [0.7, 0.7, 0.7, 0.7, 0.9], [0.7, 0.7, 0.7, 0.7, 0.9],
              [0.7, 0.7, 0.7, 0.7, 0.9]])

In [14]:
def create_arrays():
    dude = []
    array = [.9,.9,.7,.1,.1]
    for i in range(50):
        dude.append(array[:])
    return dude
        

In [16]:
create_series(create_arrays())